In [1]:
import requests
import pandas as pd
import json
import numpy as np

In [2]:
session = requests.session()

In [3]:
url = 'https://users.premierleague.com/accounts/login/'

payload = {
 'password': 'PASSWORD',
 'login': 'LOGIN',
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}

session.post(url, data=payload)

<Response [200]>

In [4]:
response = session.get(
    'https://fantasy.premierleague.com/api/my-team/{}/'.format('TEAM_ID'))

In [5]:
response.status_code

200

In [6]:
picks = pd.DataFrame(response.json()['picks']).sort_values(by='selling_price', ascending=False)

In [7]:
picks

,element,position,selling_price,multiplier,purchase_price,is_captain,is_vice_captain
14,303,15,88,0,90,False,False
9,166,10,78,2,80,True,False
10,118,11,72,1,75,False,False
13,141,14,71,0,70,False,False
6,287,7,69,1,70,False,False
7,150,8,68,1,70,False,False
8,120,9,68,1,70,False,False
5,392,6,67,1,70,False,False
2,270,3,60,1,60,False,False
3,274,4,59,1,60,False,False


In [28]:
player_url = 'https://fantasy.premierleague.com/api/element-summary/{}/'

response = session.get(player_url.format('291'))

In [30]:
one_element_id = str(picks.loc[0, 'element'])

el_resp = session.get(player_url.format(one_element_id))

In [31]:
el_resp.json().keys()

dict_keys(['fixtures', 'history', 'history_past'])

In [32]:
fixtures = pd.DataFrame(el_resp.json()['fixtures'])
history = pd.DataFrame(el_resp.json()['history'])
history_past = pd.DataFrame(el_resp.json()['history_past'])

In [33]:
# Note there are 538 elements in total

In [34]:
history_past.element_code

0    51940
1    51940
2    51940
3    51940
4    51940
5    51940
6    51940
7    51940
8    51940
Name: element_code, dtype: int64

In [14]:
team_results = {}

for el in picks['element'].unique():
    response = session.get(player_url.format(str(el)))
    if response.status_code == 200:

        fixtures = pd.DataFrame(response.json()['fixtures'])
        history = pd.DataFrame(response.json()['history'])
        history_past = pd.DataFrame(response.json()['history_past'])
        
        if not history_past.empty:
            mean_points = history_past['total_points'].mean()
            stdev = history_past['total_points'].std()
            points_range = max(history_past['total_points']) - min(history_past['total_points'])

            team_results[el] = {
                'mean': mean_points, 
                'stddev': stdev,
                'range': points_range
            }
        else:
            team_results[el] = {
                'mean': np.nan,
                'stddev': np.nan,
                'range': np.nan
            }

In [15]:
team_results

{303: {'mean': 136.6, 'stddev': 39.10626548265636, 'range': 105},
 166: {'mean': 147.66666666666666, 'stddev': 20.526405757787536, 'range': 40},
 118: {'mean': 60.25, 'stddev': 73.68570643844933, 'range': 153},
 141: {'mean': 106.42857142857143, 'stddev': 45.09936120337383, 'range': 128},
 287: {'mean': nan, 'stddev': nan, 'range': nan},
 150: {'mean': 124.11111111111111, 'stddev': 44.287256757572045, 'range': 110},
 120: {'mean': 137.0, 'stddev': nan, 'range': 0},
 392: {'mean': 80.66666666666667, 'stddev': 62.29232162420448, 'range': 120},
 270: {'mean': 100.63636363636364, 'stddev': 49.52226312936986, 'range': 143},
 274: {'mean': 51.0, 'stddev': 27.874719729532707, 'range': 54},
 457: {'mean': 78.0, 'stddev': 89.98147957589198, 'range': 167},
 104: {'mean': 96.57142857142857, 'stddev': 73.1912042594763, 'range': 170},
 291: {'mean': 136.33333333333334, 'stddev': 16.55294535724685, 'range': 58},
 81: {'mean': 91.0, 'stddev': 49.879855653359705, 'range': 120},
 305: {'mean': 160.0, '

In [16]:
results = pd.DataFrame.from_dict(team_results).T.sort_values(by='mean', ascending=False)

In [17]:
results

,mean,stddev,range
305,160.000000,NaN,0.0
166,147.666667,20.526406,40.0
120,137.000000,NaN,0.0
303,136.600000,39.106265,105.0
291,136.333333,16.552945,58.0
150,124.111111,44.287257,110.0
141,106.428571,45.099361,128.0
270,100.636364,49.522263,143.0
104,96.571429,73.191204,170.0
81,91.000000,49.879856,120.0


In [39]:
master_data = session.get('https://fantasy.premierleague.com/api/bootstrap-static/')
players = master_data.json()['elements']
players_df = pd.DataFrame(players)
players_df.set_index('id', inplace=True)

In [40]:
players_df

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,37605,0,0,-3,3,0,3,0.0,...,642,258,642,258,NaN,,NaN,,NaN,
2,0.0,0.0,39476,0,0,-2,2,0,2,0.0,...,583,215,603,219,NaN,,NaN,,NaN,
3,100.0,100.0,41270,0,0,-1,1,0,2,2.6,...,253,77,300,99,NaN,,4.0,,NaN,
4,0.0,0.0,54694,0,0,-7,7,1,3,0.0,...,22,13,34,22,NaN,,NaN,,1.0,
5,100.0,100.0,58822,0,0,-4,4,0,2,2.8,...,372,139,356,130,NaN,,NaN,,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,NaN,NaN,481626,0,0,0,0,0,2,0.0,...,460,180,510,196,NaN,,NaN,,NaN,
617,NaN,NaN,448487,0,0,0,0,0,1,0.0,...,456,23,506,53,NaN,,NaN,,NaN,
626,NaN,NaN,209353,-1,1,-1,1,0,4,0.7,...,396,60,466,61,NaN,,NaN,,NaN,


In [38]:
players_df[players_df['second_name'] == 'Sigurdsson']['id']

193    150
Name: id, dtype: int64

In [44]:
players_df.shape

(643, 66)

In [22]:
history_past['total_points'].mean()

160.0

In [23]:
mean_points = history_past['total_points'].mean()
stdev = history_past['total_points'].std()
points_range = max(history_past['total_points']) - min(history_past['total_points'])

team_results[one_element_id] = {
    'mean': mean_points, 
    'stddev': stdev,
    'range': points_range
}

In [24]:
response.status_code

200

In [25]:
pd.DataFrame(response.json()['fixtures'])

,id,code,team_h,team_h_score,team_a,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty
0,192,2128479,13,None,15,None,20,False,0,False,2021-01-27T20:15:00Z,Gameweek 20,True,2
1,199,2128486,1,None,13,None,21,False,0,False,2021-01-30T17:30:00Z,Gameweek 21,False,3
2,213,2128500,13,None,16,None,22,False,0,False,2021-02-02T20:15:00Z,Gameweek 22,True,3
3,224,2128511,13,None,7,None,23,False,0,False,2021-02-08T20:00:00Z,Gameweek 23,True,3
4,237,2128524,18,None,13,None,24,False,0,False,2021-02-13T15:00:00Z,Gameweek 24,False,2
5,245,2128532,13,None,14,None,25,False,0,False,2021-02-21T19:00:00Z,Gameweek 25,True,2
6,249,2128536,5,None,13,None,26,False,0,False,2021-02-28T16:30:00Z,Gameweek 26,False,4
7,264,2128551,12,None,13,None,27,False,0,False,2021-03-06T15:00:00Z,Gameweek 27,False,5
8,275,2128562,13,None,19,None,28,False,0,False,2021-03-13T15:00:00Z,Gameweek 28,True,3
9,281,2128568,6,None,13,None,29,False,0,False,2021-03-20T15:00:00Z,Gameweek 29,False,3


In [26]:
for i in range(500):
    response = session.get(player_url.format(str(i)))
    if response.status_code != 200:
        print(i)

0


KeyboardInterrupt: 